# Reddit stock market prediction

### Import libraries

In [ ]:
# pull libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')

### Preprocess data

In [ ]:
# pull data
djia_data = pd.read_csv('./stocknews/Combined_News_DJIA.csv', parse_dates=True)

djia_data['combined'] = djia_data[djia_data.columns[2:]].apply(
    lambda x: ' '.join(x.astype(str)),
    axis=1
)

def sanitise_row(row):
    return re.sub('[^A-Za-z ]+', '', row.replace("b\"", "").replace("b'", ""))

def remove_stop_words(sentence):
    sentence = sanitise_row(sentence)
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

djia_data['combined'] = djia_data['combined'].map(lambda x: remove_stop_words(x))

# print(type(djia_data['combined']))

In [ ]:
# Ratio of label 1s and 0s
data_label = np.array(djia_data['Label'])

print(np.sum(data_label == 1) / data_label.shape[0])

### Preprocess data

In [ ]:
# prepare train and test data
all_data = djia_data['combined']
data_label = np.array(djia_data['Label'])

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(all_data)
# print(vectorizer.get_feature_names())
# print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    all_data, data_label, test_size=0.50, random_state=23)
# we also need dictionary that maps word to number of occurences -> 
# to handle cases where word is not found in the training corpus

In [ ]:
# We need to convert training data into a vector for LSTM
# how does the vector look like?
# need to build word_index first!


# generate word numbers
def gen_word_indexes(data): 
    word_index = {}
    word_index["<PAD>"] = 0
    word_index["<START>"] = 1
    word_index["<UNK>"] = 2  # unknown
    word_index["<UNUSED>"] = 3

    index = 4
    # loop through everything in all_data
    for row in data:
        for word in row.split():
            word = word.lower()
            if word not in word_index:
                word_index[word] = index
                index += 1

    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    return (word_index, reverse_word_index)


# text --> numbers corresponding to words
def vectorise_text_data(data, word_index):
    vectorised_data = []
    for row in data:
        current_row = []
        for word in row.split():
            word = word.lower()
            current_row.append(word_index[word])
        vectorised_data.append(current_row)
    return vectorised_data

#
def aggregate_previous_days(num_days, data):
    for i in range(num_days, len(data) - num_days):
        for d in range(i, i + num_days):
            data[i] += ' ' + data[d]
    return data    

In [25]:
(word_index, reverse_word_index) = gen_word_indexes(all_data)

# X5day_vectorised = pd.vectorise_text_data(time_series_data, word_index)


In [26]:
time_series_data = aggregate_previous_days(5, all_data)
print(time_series_data)

{'<PAD>': 0, '<START>': 1, '<UNK>': 2, '<UNUSED>': 3, 'georgia': 4, 'downs': 5, 'two': 6, 'russian': 7, 'warplanes': 8, 'countries': 9, 'move': 10, 'brink': 11, 'war': 12, 'breaking': 13, 'musharraf': 14, 'impeached': 15, 'russia': 16, 'today': 17, 'columns': 18, 'troops': 19, 'roll': 20, 'south': 21, 'ossetia': 22, 'footage': 23, 'fighting': 24, 'youtube': 25, 'tanks': 26, 'moving': 27, 'towards': 28, 'capital': 29, 'reportedly': 30, 'completely': 31, 'destroyed': 32, 'georgian': 33, 'artillery': 34, 'fire': 35, 'afghan': 36, 'children': 37, 'raped': 38, 'impunity': 39, 'un': 40, 'official': 41, 'says': 42, 'sick': 43, 'three': 44, 'year': 45, 'old': 46, 'nothing': 47, 'entered': 48, 'whilst': 49, 'shoots': 50, 'jets': 51, 'invades': 52, 'warned': 53, 'would': 54, 'intervene': 55, 'sos': 56, 'side': 57, 'the': 58, 'enemy': 59, 'combatent': 60, 'trials': 61, 'sham': 62, 'salim': 63, 'haman': 64, 'sentenced': 65, 'years': 66, 'kept': 67, 'longer': 68, 'anyway': 69, 'feel': 70, 'like': 7

In [ ]:
from keras.preprocessing import sequence

# number of most-frequent words to use
nb_words = 40000
# cut texts after this number of words
maxlen = 100

# we can change the "X???_vectorised" based on the number of cumulative days:
X_v_train, X_v_test, y_v_train, y_v_test = train_test_split(
    X_vectorised, data_label, test_size=0.20, random_state=23)

X_v_pad_train = sequence.pad_sequences(X_v_train,
                                       value=word_index["<PAD>"],
                                       padding='post',
                                       maxlen=maxlen)
X_v_pad_test = sequence.pad_sequences(X_v_test, 
                                      value=word_index["<PAD>"],
                                      padding='post',
                                      maxlen=maxlen)

print(X_v_pad_train.shape, X_v_pad_test.shape)
print(X_v_pad_train[2])

Now what? We train!

### Training using Naive Bayes

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# LogisticRegression(random_state=0, solver='lbfgs')

v = CountVectorizer()

text_clf = Pipeline([
    ('vect', v),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)  

### Training using LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM

EMBEDDING_DIM = 128 # dimension for dense embeddings for each token
LSTM_DIM = 64 # total LSTM units

model = Sequential()
model.add(Embedding(input_dim=nb_words, output_dim=EMBEDDING_DIM, input_length=maxlen))
model.add(SpatialDropout1D(0.8))
model.add(LSTM(LSTM_DIM, dropout=0.8, recurrent_dropout=0.8))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [163]:
batch_size = 128
model.fit(X_v_pad_train, y_v_train, epochs=6, batch_size=batch_size, 
          shuffle=True, validation_split=0.4, verbose=1)

Train on 954 samples, validate on 637 samples
Epoch 1/6
954/954 [==============================] - 8s 8ms/step - loss: 0.6943 - acc: 0.5021 - val_loss: 0.6929 - val_acc: 0.5118
Epoch 2/6
954/954 [==============================] - 3s 3ms/step - loss: 0.6933 - acc: 0.5126 - val_loss: 0.6927 - val_acc: 0.5149
Epoch 3/6
954/954 [==============================] - 3s 3ms/step - loss: 0.6935 - acc: 0.5210 - val_loss: 0.6926 - val_acc: 0.5181
Epoch 4/6
954/954 [==============================] - 3s 3ms/step - loss: 0.6929 - acc: 0.5314 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 5/6
954/954 [==============================] - 3s 3ms/step - loss: 0.6891 - acc: 0.5430 - val_loss: 0.6925 - val_acc: 0.5181
Epoch 6/6
954/954 [==============================] - 3s 3ms/step - loss: 0.6905 - acc: 0.5440 - val_loss: 0.6924 - val_acc: 0.5181


In [164]:
scores = model.evaluate(X_v_pad_test, y_v_test, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 56.28%
